In [1]:
from transformers import BertForMaskedLM, BertTokenizer,Trainer,DataCollatorForLanguageModeling


tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForMaskedLM.from_pretrained('beomi/kcbert-base')

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from datasets import load_dataset

train = load_dataset('csv',data_files='data/book_train_128.csv')
validation = load_dataset('csv',data_files='data/book_validation_128.csv')

In [188]:
def tokenize_function(examples):
    return tokenizer(examples['sen'], max_length=128, padding=True, truncation=True)

train_data_set = train['train'].map(tokenize_function,batch_size=True)
validation_data_set = validation['train'].map(tokenize_function,batch_size=True)

Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-b37cd5b9b58c5c8c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-5cfc02813bdf83b5.arrow
Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-df0bc971df98e82e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-bf7c412741a4be5e.arrow


In [197]:
from transformers import TrainingArguments, TrainerCallback

### Training Arguments
device = 'cpu'

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    logging_steps=100,
    num_train_epochs=2,
)

data_collator_BERT = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15, return_tensors="pt"
)


class myCallback(TrainerCallback):
    """
    A bare :class:`~transformers.TrainerCallback` that just prints the logs.
    """

    def on_step_begin(self, args, state, control, logs=None, **kwargs):

        # 함수 이름.. 언제 시작할지
        # log는 설정할 때마다
        # arg,state,control은 참고할 수 있는 attribute의 경우임.
        # 근데 내가 필요한건 input
        if state.global_step % args.logging_steps == 0:
            print("")
            print(
                f"{int(state.epoch)}번째 epoch 진행 중 ------- {state.global_step}번째 step 결과"
            )


class customtrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    ############# 내용 추가
    def step_check(self):
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            # step_check = 현 step 파악
            # args.logging_steps = argument에서 지정한 step 불러오가

            # batch 중 0 번째 위치한 문장 선택
            num = 1
            input_id = inputs.input_ids[num].reshape(-1).data.tolist()
            output_id = outputs.logits[num].argmax(dim=-1).reshape(-1).data.tolist()
            attention_mask = inputs.attention_mask[num]

            # mask가 위치한 idx 추출하기
            mask_idx = (inputs.input_ids[num] == 4).nonzero().data.reshape(-1).tolist()

            # padding 제거
            input_id_without_pad = [
                input_id[i] for i in range(len(input_id)) if attention_mask[i]
            ]
            output_id_without_pad = [
                output_id[i] for i in range(len(output_id)) if attention_mask[i]
            ]

            # id to token
            # [1:-1] [CLS,SEP] 제거
            inputs_tokens = self.tokenizer.convert_ids_to_tokens(input_id_without_pad)[
                1:-1
            ]
            outputs_tokens = self.tokenizer.convert_ids_to_tokens(
                output_id_without_pad
            )[1:-1]

            # output mask 부분 표시하기
            for i in mask_idx:
                # [CLS,SEP 위치 조정]
                outputs_tokens[i - 1] = "[" + outputs_tokens[i - 1] + "]"

            inputs_sen = self.tokenizer.convert_tokens_to_string(inputs_tokens)
            outputs_sen = self.tokenizer.convert_tokens_to_string(outputs_tokens)

            print(f"input 문장 : {''.join(inputs_sen)}")
            print(f"output 문장 : {''.join(outputs_sen)}")

        return (loss, outputs) if return_outputs else loss


trainer = customtrainer(
    model=model.to(device),
    train_dataset=train_data_set,
    eval_dataset=validation_data_set,
    data_collator=data_collator_BERT,
    args=training_args,
    tokenizer=tokenizer,
    callbacks=[myCallback],
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: Unnamed: 0, sen. If Unnamed: 0, sen are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 175900
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21988


  0%|          | 0/21988 [00:00<?, ?it/s]


0번째 epoch 진행 중 ------- 0번째 step 결과
input 문장 : 장 수학 기호 수식에 많이 쓰이는 그리스 알 [MASK] [MASK] [MASK] 읽고 쓰는 법을 배웁니다
output 문장 : 장 수학 기호 수식에 많이 쓰이는 그리스 알 [##파] [##벳] [##을]을 쓰는 법을 배웁니다
{'loss': 0.0233, 'learning_rate': 4.99886301619065e-05, 'epoch': 0.0}

0번째 epoch 진행 중 ------- 5번째 step 결과
input 문장 : 물론 이들 언어로 클라이언트 측 [MASK]리케이션을 작성할 [MASK] 있다
output 문장 : 물론 이들 언어로 클라이언트 측 [애플]리케이션을 작성할 [수도] 있다
{'loss': 0.0166, 'learning_rate': 4.997726032381299e-05, 'epoch': 0.0}

0번째 epoch 진행 중 ------- 10번째 step 결과
input 문장 : [MASK] 사이 PHP는 Jav [MASK]라고 불리는 Java8처럼 또 최근의 많은 언어가 그러하듯 여러 개발 언어의 좋은 장점이나 [MASK]들을 차용하여 적극 도입하고 개선하여 Modern PHP로 거듭났고 재 [MASK] 중인 회사는 여전히 PHP가 주 [MASK] 개발 언어로 사용 해 [MASK]습니다 .
output 문장 : [그] 사이 PHP는 Jav [##a]라고 불리는 Java8처럼 또 최근의 많은 언어가 그러하듯 여러 개발 언어의 좋은 장점이나 [기능]들을 차용하여 적극 사용하고 개선하여 Modven PHP로 거듭났고 재 [##직] 중인 회사는 여전히 PHP가 주 [##력] 개발 언어로 사용 해 [##왔]습니다 .
{'loss': 0.765, 'learning_rate': 4.9965890485719485e-05, 'epoch': 0.0}

0번째 epoch 진행 중 ------- 15번째 step 결과
input 문장 : 아무래도 개념부터 확실히 [M

KeyboardInterrupt: 

In [186]:
class customtrainer(Trainer) :
    def __init__(self, *args,**kwargs):
        super().__init__(*args, **kwargs)
    
    ############# 내용 추가
    def step_check(self) :
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        
        ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            # step_check = 현 step 파악
            # args.logging_steps = argument에서 지정한 step 불러오가

            # batch 중 0 번째 위치한 문장 선택
            num = 1
            input_id = inputs.input_ids[num].reshape(-1).data.tolist()
            output_id = outputs.logits[num].argmax(dim=-1).reshape(-1).data.tolist()
            attention_mask = inputs.attention_mask[num]

            # mask가 위치한 idx 추출하기 
            mask_idx = (inputs.input_ids[num] == 4).nonzero().data.reshape(-1).tolist()


            # padding 제거
            input_id_without_pad = [input_id[i] for i in range(len(input_id)) if attention_mask[i]]
            output_id_without_pad = [output_id[i] for i in range(len(output_id)) if attention_mask[i]]

            # id to token 
            # [1:-1] [CLS,SEP] 제거
            inputs_tokens = self.tokenizer.convert_ids_to_tokens(input_id_without_pad)[1:-1]
            outputs_tokens = self.tokenizer.convert_ids_to_tokens(output_id_without_pad)[1:-1]

            # output mask 부분 표시하기
            for i in mask_idx :
                # [CLS,SEP 위치 조정]
                outputs_tokens[i-1] = '[' + outputs_tokens[i-1] + ']'


            inputs_sen = self.tokenizer.convert_tokens_to_string(inputs_tokens)
            outputs_sen = self.tokenizer.convert_tokens_to_string(outputs_tokens)


            print(f"input 문장 : {''.join(inputs_sen)}"  )
            print(f"output 문장 : {''.join(outputs_sen)}"  )
            

        return (loss, outputs) if return_outputs else loss

trainer = customtrainer(model=model, train_dataset=train_data_set, eval_dataset=validation_data_set,data_collator=data_collator_BERT,args=training_args,tokenizer=tokenizer)

trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: Unnamed: 0, sen. If Unnamed: 0, sen are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 5


  0%|          | 0/5 [00:00<?, ?it/s]

0번째 step 결과
input 문장 : [MASK] PCI PCIe 디 [MASK]스 드라이버 에서는 PCI와 PCIe PCI델 [MASK] [MASK]ress 버스를 소개하고 이를 사용하는 카드들을 [MASK] 윈도우 드라이버 작성법을 [MASK]하고 있다
output 문장 : [장] PCI PCIe 디 [##바이]스 드라이버 에서는 PCI와 PCIe PCI E [##x] [##p]ress 버스를 소개하고 이를 사용하는 카드들을 [위한] 윈도우 드라이버 작성법을 [소개]하고 있다




Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 0.0, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 2.4029, 'train_samples_per_second': 4.162, 'train_steps_per_second': 2.081, 'train_loss': 2.7520098228706045e-06, 'epoch': 5.0}


TrainOutput(global_step=5, training_loss=2.7520098228706045e-06, metrics={'train_runtime': 2.4029, 'train_samples_per_second': 4.162, 'train_steps_per_second': 2.081, 'train_loss': 2.7520098228706045e-06, 'epoch': 5.0})